Il k-nearest neighbors è un algoritmo di classificazione basato sull'assunto che le osservazioni simili appartengano alla stessa classe. Consiste nel calcolare la distanza tra un'osservazione di test e tutte le osservazioni del dataset di training e nell'assegnare l'etichetta della classe più presente tra i k osservazioni più vicine.

Il processo è questo:

1) Scegliere il numero di vicini k da considerare durante il calcolo delle distanze

2) Calcolare la distanza tra l'osservazione di test e tutte le osservazioni del dataset di training

3) Selezionare le k osservazioni più vicine

4) Assegnare l'etichetta della classe più presente tra le k osservazioni più vicine all'osservazione di test

Questo algoritmo può essere lento nel caso di dataset di grandi dimensioni e l'accuratezza del modello può essere influenzata in modo significativo dalla scelta del numero di vicini k.


Pro: 

Semplicità: è un algoritmo semplice e facile da capire e implementare

Buona prestazione con dataset di piccole dimensioni

Contro:

Lentezza con dataset di grandi dimensioni perchè deve calcolare la distanza tra l'osservazione di test e tutte le osservazioni ogni volta. Infatti è considerato un algoritmo lazy learner, ossia un algoritmo che richiede meno tempo per l'apprendimento e più tempo per la classificazione

Sensibilità alla scelta del numero k di vicini

Sensibilità alla presenza di outliers

In [11]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)


options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")
df <- df_finale

# Feature selection

Feature selection fatta attraverso l'algoritmo boruta che si basa sull'algoritmo di random forest e utilizza una tecnica di confronto dei risultati ottenuti con un insieme di dati di controllo per determinare quali variabili sono significative. è un algoritmo che tiene conto delle interazioni tra variabili, è versatile in quanto lavora sia su variabili continue che categoriche ed è resistente agli outliers

In [12]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +1.1 mins: 

 confirmed 4 attributes: centreCode, IntraoperativTransfected, realMELD, surgeryDuration;

 rejected 49 attributes: abdomOpenEndSurg, age, albumina, albumInfusOver7Days, anastomTechnic and 44 more;

 still have 24 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +1.2 mins: 

 rejected 3 attributes: coldIschemia, organDonorRecondOrgan, organDonorSampleTechnic;

 still ha

 [1] "centreCode"               "presentRefractAscites"   
 [3] "Sodiemia"                 "ratioPaO2FiO2"           
 [5] "platCountTranspl"         "surgeryDuration"         
 [7] "IntraoperativTransfected" "plasmaFresco"            
 [9] "plasmaPools"              "doseFibrinogen"          
[11] "Albumina50Comp"           "scoreCHILD"              
[13] "organDonorType"           "regionalPerfusion"       
[15] "organDonorDrawalSodiemia" "organDonorGammaGT"       
[17] "realMELD"                 "organDonorMaastricht"    
[19] "tumore"                  


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [13]:
plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")

In [14]:
df_boruta$deg_cat <- df$deg_cat

## Data partition

Viene effettuata divisione tra training e test set, che è del 70-30 per tutti i modelli utilizzati

In [15]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)



Viene effettuato un bilanciamento della variabile target perchè in origine la classe ottimo conteneva circa 1/4 dei record rispetto alle altre due classi e i modelli facevano fatica a classificare quel determinato valore. La funzione utilizzata è SmoteClassif() che:

1) Seleziona a caso una osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

2) Seleziona a caso un'altra osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

3) Genera una nuova osservazione sintetica che si trova a metà strada tra le due osservazioni selezionate in precedenza

4) Ripete i passaggi precedenti fino a raggiungere il bilanciamento desiderato


Questa funzione utilizza il knn per calcolare le distanze tra le osservazioni e generare le osservazioni, infatti il valore di k è stato selezionato pari alla radice quadrata del numero totale di record presenti nel training set come si fa per il knn normalmente per il knn



In [16]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    370     370 

Viene effettuato lo stesso processo considerando il dataset dopo la boruta feature selection invece del dataset con tutte le feature in modo da poter fare una comparazione sui risultati successivamente

In [17]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_boruta$deg_cat, SplitRatio = 0.7)
trainB  <- subset(df_boruta, sample == TRUE)
testB   <- subset(df_boruta, sample == FALSE)



TestClassesB <- testB$deg_cat
TestDataB <- subset(testB,select= -deg_cat)


In [18]:
trainB <- SmoteClassif(deg_cat ~ ., trainB, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(trainB$deg_cat)

TrainClassesB <- trainB$deg_cat
TrainDataB <- subset(trainB,select= -deg_cat)


   fast NotFast 
    370     370 

## Models

Attraverso la funzione trainControl viene effettuata una repeated cross validation e una random search.

La prima permette di ottenere una stima più accurata dell'accuratezza del modello:

1) Dividere il dataset in un numero predeterminato di folds (in questo caso è stato scelto n=10 quindi 10 folds)

2) Eseguire la cross-validation ripetutamente utilizzando ogni fold come test set e gli altri folds come training set

3) Calcolare la media e la deviazione standard delle misure di accuratezza ottenute per ogni fold

4) Valutare l'accuratezza e la robustezza del modello utilizzando la media e la deviazione standard delle misure di accuratezza

L'utilizzo della repeated cross validation è particolarmente importante quando si lavora con dataset di dimensioni ridotte o con un numero limitato di osservazioni perchè in questi casi i risultati possono essere influenzata dall'aleatorietà nella divisione dei dati in training set e test set. La repeated cross-validation permette quindi di ottenere una stima più accurata dell'accuratezza del modello ripetendo il processo  più volte e considerando dati diversi



La random search, invece è una tecnica di ottimizzazione dei modelli che viene utilizzata per selezionare i migliori parametri di un modello. Consiste nel generare a caso diverse combinazioni di parametri e nel valutare l'accuratezza del modello per ogni combinazione. La combinazione di parametri che ottiene il risultato migliore viene quindi selezionata come quella ottimale. Questa soluzione è stata preferita alla grid search che poteva dare risultati migliori per una questione di costo computazionale

In [19]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

Viene allenato il modello knn sul dataset senza feature selection

In [20]:
model_knn <- train(x=TrainData, y=TrainClasses, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax=231, distance=0.4878, kernel=biweight 
- Fold01.Rep1: kmax=231, distance=0.4878, kernel=biweight 
+ Fold01.Rep1: kmax= 59, distance=1.9017, kernel=triweight 
- Fold01.Rep1: kmax= 59, distance=1.9017, kernel=triweight 
+ Fold01.Rep1: kmax= 89, distance=1.7336, kernel=biweight 
- Fold01.Rep1: kmax= 89, distance=1.7336, kernel=biweight 
+ Fold02.Rep1: kmax=231, distance=0.4878, kernel=biweight 
- Fold02.Rep1: kmax=231, distance=0.4878, kernel=biweight 
+ Fold02.Rep1: kmax= 59, distance=1.9017, kernel=triweight 
- Fold02.Rep1: kmax= 59, distance=1.9017, kernel=triweight 
+ Fold02.Rep1: kmax= 89, distance=1.7336, kernel=biweight 
- Fold02.Rep1: kmax= 89, distance=1.7336, kernel=biweight 
+ Fold03.Rep1: kmax=231, distance=0.4878, kernel=biweight 
- Fold03.Rep1: kmax=231, distance=0.4878, kernel=biweight 
+ Fold03.Rep1: kmax= 59, distance=1.9017, kernel=triweight 
- Fold03.Rep1: kmax= 59, distance=1.9017, kernel=triweight 
+ Fold03.Rep1: kmax= 89, distance=1.7336, kernel=b

Viene allenato il modello knn sul dataset con feature selection

In [21]:
model_knnB <- train(x=TrainDataB, y=TrainClassesB, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax= 85, distance=2.5449, kernel=epanechnikov 
- Fold01.Rep1: kmax= 85, distance=2.5449, kernel=epanechnikov 
+ Fold01.Rep1: kmax=205, distance=1.8396, kernel=triangular 
- Fold01.Rep1: kmax=205, distance=1.8396, kernel=triangular 
+ Fold01.Rep1: kmax=139, distance=0.3013, kernel=gaussian 
- Fold01.Rep1: kmax=139, distance=0.3013, kernel=gaussian 
+ Fold02.Rep1: kmax= 85, distance=2.5449, kernel=epanechnikov 
- Fold02.Rep1: kmax= 85, distance=2.5449, kernel=epanechnikov 
+ Fold02.Rep1: kmax=205, distance=1.8396, kernel=triangular 
- Fold02.Rep1: kmax=205, distance=1.8396, kernel=triangular 
+ Fold02.Rep1: kmax=139, distance=0.3013, kernel=gaussian 
- Fold02.Rep1: kmax=139, distance=0.3013, kernel=gaussian 
+ Fold03.Rep1: kmax= 85, distance=2.5449, kernel=epanechnikov 
- Fold03.Rep1: kmax= 85, distance=2.5449, kernel=epanechnikov 
+ Fold03.Rep1: kmax=205, distance=1.8396, kernel=triangular 
- Fold03.Rep1: kmax=205, distance=1.8396, kernel=triangular 
+ Fold03.Rep1: kmax=

I parametri testati sono: 

kmax = numero di nearest neighbors considerati

distance = parametro per la distanza minkowski

kernel = per specificare il metodo di con cui vengono calcolati i pesi da utilizzare durante il calcolo delle distanze tra le osservazioni

## Results

Vengono analizzati i risultati ottenuti dai due modelli durante la cross validation. Le metriche utilizzate sono: accuracy e kappa. La prima si riferisce alla percentuale di osservazioni correttamente classificate dal modello rispetto al totale di osservazioni. La seconda viene utilizzata per misurare l'accordo tra più modelli di classificazione. Può assumere valori compresi tra -1 e 1. Un valore di kappa vicino a 1 indica un alto livello di accordo tra le classificazioni, mentre un valore di kappa vicino a 0 indica un accordo simile a quello ottenuto per caso. Un valore di kappa negativo indica un accordo peggiore di quello ottenuto per caso.

In [22]:
print(model_knn)
plot(model_knn)

k-Nearest Neighbors 

740 samples
 77 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 666, 666, 666, 666, 666, 666, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel     Accuracy   Kappa    
   59   1.9016631  triweight  0.7972973  0.5945946
   89   1.7335631  biweight   0.8013514  0.6027027
  231   0.4878169  biweight   0.7972973  0.5945946

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 89, distance = 1.733563
 and kernel = biweight.


In [23]:
print(model_knnB)
plot(model_knnB)

k-Nearest Neighbors 

740 samples
 19 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 666, 666, 666, 666, 666, 666, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel        Accuracy   Kappa    
   85   2.5448926  epanechnikov  0.8648649  0.7297297
  139   0.3012557  gaussian      0.8878378  0.7756757
  205   1.8396230  triangular    0.8797297  0.7594595

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 139, distance = 0.3012557
 and kernel = gaussian.


Vengono messi a confronto i risultati ottenuti sulla cross validation dei due modelli in modo da selezionare il più performante da utilizzare nel test set

In [24]:
# collect resamples
results <- resamples(list(knn=model_knn,knn_boruta=model_knnB))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: knn, knn_boruta 
Number of resamples: 10 

Accuracy 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.7162162 0.7972973 0.8040541 0.8013514 0.8209459 0.8513514    0
knn_boruta 0.8378378 0.8547297 0.8851351 0.8878378 0.9054054 0.9729730    0

Kappa 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.4324324 0.5945946 0.6081081 0.6027027 0.6418919 0.7027027    0
knn_boruta 0.6756757 0.7094595 0.7702703 0.7756757 0.8108108 0.9459459    0


## Prediction

Viene selezionato il modello più performante per usarlo nel test set ed avere risultati definitivi.

In [25]:
pred_knn <- predict(model_knnB,test)  
pred_knn.prob <- predict(model_knnB,test, type="prob")


result <- confusionMatrix(TestClasses,  pred_knn) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      24      18
   NotFast   53     223
                                          
               Accuracy : 0.7767          
                 95% CI : (0.7269, 0.8213)
    No Information Rate : 0.7579          
    P-Value [Acc > NIR] : 0.2375          
                                          
                  Kappa : 0.2804          
                                          
 Mcnemar's Test P-Value : 5.459e-05       
                                          
            Sensitivity : 0.31169         
            Specificity : 0.92531         
         Pos Pred Value : 0.57143         
         Neg Pred Value : 0.80797         
             Prevalence : 0.24214         
         Detection Rate : 0.07547         
   Detection Prevalence : 0.13208         
      Balanced Accuracy : 0.61850         
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.3116883            0.9253112            0.5714286 
      Neg Pred Value            Precision               Recall 
           0.8079710            0.5714286            0.3116883 
                  F1           Prevalence       Detection Rate 
           0.4033613            0.2421384            0.0754717 
Detection Prevalence    Balanced Accuracy 
           0.1320755            0.6184998

In [26]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_knn))
roc_knn



Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6897

In [27]:
roc_obj <- roc(TestClasses, as.numeric(pred_knn))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [28]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) in 42 controls (TestClasses fast) < 276 cases (TestClasses NotFast).
Area under the curve: 0.6897

Probabilità per ogni classe

In [29]:
pred_knn.prob

fast,NotFast
0.0000000,1.0000000
0.0000000,1.0000000
0.2208688,0.7791312
0.0000000,1.0000000
0.2304652,0.7695348
0.4778299,0.5221701
0.2379062,0.7620938
0.2001349,0.7998651
0.7528470,0.2471530
0.3060572,0.6939428
